# Chapter 1: Prompt Chaining (代码示例)

本 notebook 演示了如何使用 LangChain 实现 Prompt Chaining（提示链）模式。

## 前置要求

- Python 3.8+
- 阿里云 DashScope API Key（需要设置环境变量 `DASHSCOPE_API_KEY`）

## 1. 安装依赖

首先安装所需的 Python 包。

In [8]:
# 安装必要的依赖包（使用 %pip 在 Jupyter 中更推荐）
%pip install -q langchain-core>=0.1.0 langchain-community>=0.1.0 dashscope>=1.17.0 python-dotenv>=1.0.0

Note: you may need to restart the kernel to use updated packages.


## 2. 获取 API Key

如果环境变量或 .env 文件中没有设置 `DASHSCOPE_API_KEY`，可以在此处直接设置。

In [9]:
import os
from dotenv import load_dotenv
load_dotenv()

# 获取 API key（优先从环境变量读取，如果没有则从 .env 文件）
dashscope_api_key = os.getenv("DASHSCOPE_API_KEY")

## 3. 导入库并初始化模型

In [10]:
from langchain_community.chat_models import ChatTongyi
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


# 初始化 Qwen 语言模型（使用阿里云 DashScope API）
# model 参数可以指定不同的 Qwen 模型，如 "qwen-flash", "qwen-turbo", "qwen-plus", "qwen-max" 等
llm = ChatTongyi(
    model="qwen-flash",
    temperature=0,
    dashscope_api_key=dashscope_api_key
)

# --- Prompt 1: 提取信息 ---
prompt_extract = ChatPromptTemplate.from_template(
    "从以下文本中提取技术规格：\n\n{text_input}"
)

# --- Prompt 2: 转换为 JSON ---
prompt_transform = ChatPromptTemplate.from_template(
    "将以下规格转换为 JSON 对象，包含 'cpu'、'memory' 和 'storage' 作为键：\n\n{specifications}"
)

# --- 使用 LCEL 构建链 ---
# StrOutputParser() 将 LLM 的消息输出转换为简单字符串
extraction_chain = prompt_extract | llm | StrOutputParser()

# 完整链将提取链的输出传递给转换提示的 'specifications' 变量
full_chain = (
    {"specifications": extraction_chain}
    | prompt_transform
    | llm
    | StrOutputParser()
)

## 4. 运行提示链

现在让我们执行提示链来处理输入文本。

In [11]:
# --- 运行链 ---
input_text = "新款笔记本电脑配备 3.5 GHz 八核处理器、16GB RAM 和 1TB NVMe SSD。"

# 使用输入文本字典执行链
final_result = full_chain.invoke({"text_input": input_text})

print("\n--- 最终 JSON 输出 ---")
print(final_result)


--- 最终 JSON 输出 ---
```json
{
  "cpu": "3.5 GHz 八核处理器",
  "memory": "16GB",
  "storage": "1TB NVMe SSD"
}
```
